### 本番用メインファイルのサンプル
pyスクリプトに書き換えてオートメーションするのとnotebook形式で実行するのどっちがいいかな...。とりあえずネットワークインタフェースだけモジュール化してあとはnotebookでやってみる。

In [ ]:
# ライブラリ読み込み
import numpy as np
import matplotlib.pyplot as plt
import librosa
import glob
import json
%matplotlib inline

from typing import List
from modules.network_interface import *

sampling_rate = 30000

In [ ]:
# 音源読み込み
def load_audio(file_name: str) -> np.ndarray:
    return librosa.load(file_name, sr=sampling_rate, mono=True)[0]

In [ ]:
# 読みデータ一括読み込み
def load_audios(lang: str) -> List[np.ndarray]:
    if lang != 'J' and lang != 'E':
        print("ファイルが見つかりません")
        return []
    
    audio_dir = './JKspeech/'
    file_names = glob.glob(f'{audio_dir}/{lang}*.wav')
    return [load_audio(file_name) for file_name in file_names]

In [ ]:
# 畳み込みで適合率を求める
def get_precision(origin_init: np.ndarray, target_init: np.ndarray) -> np.ndarray:
    # サイズ調整
    start = np.abs(len(origin_init) - len(target_init)) / 180
    start = np.min([int(start), 640])
    end = np.min([len(origin_init), len(target_init)])
    target = target_init[start:end]
    origin = origin_init[start:end]

    # フーリエ変換
    fft_O = np.fft.fft(origin)
    fft_T = np.fft.fft(target)

    # 畳み込み処理
    fft_Oc = np.conj(fft_O)        # 複素共役を取る
    sigma_C = fft_Oc * fft_T       # 要素ごとに乗算
    sigma_T = np.fft.ifft(sigma_C) # 逆フーリエ変換
    sigma_T = np.abs(sigma_T)      # 絶対値を取りスカラー値に変換
    
    return np.max(sigma_T)         # スカラー値の最大値  

In [ ]:
# メタデータ読み込み
f = open('./questions_ja/metadata.json', 'r', encoding="utf-8_sig")
data = json.load(f)
f.close()

In [ ]:
sounds = load_audios('J')

In [ ]:
# problem_id = "Q016_ja"
# chunk = load_audio(f"./questions_ja/{problem_id}.wav")

In [ ]:
# precision = [get_precision(chunk, sound) for sound in sounds]

In [ ]:
# problem_id = "Q017_ja"

# metadata = data[problem_id]
# N = metadata['length']
# answer = []
# for i in np.argsort(precision)[::-1][:N]:
# #     print(i+1, result[i])
#     answer.append(i+1)

# correct_numbers = metadata['correct_numbers']
# matches = [n for n in answer if n in correct_numbers]
# shortages = [n for n in correct_numbers if n not in answer]

# print("\n問題ナンバー:", problem_id)
# print("正答:", correct_numbers)
# print("回答:", sorted(answer))
# print("一致:", sorted(matches))
# print("不足:", sorted(shortages))
# print(f"正答率: {(len(matches) / N) * 100:.02f}%")

In [ ]:
# print("Start\n")

# for _ in range(18):
#     problem_id = f"Q0{_+1:02}_ja"
#     metadata = data[problem_id]
#     N = metadata['length']
#     fname = f"./questions_ja/{problem_id}.wav"
#     chunk = load_audio(f"./questions_ja/{problem_id}.wav")

#     precision = [get_precision(chunk, sound) for sound in sounds]

#     answer = []
#     for i in np.argsort(precision)[::-1][:N]:
#         answer.append(i+1)

#     correct_numbers = metadata['correct_numbers']
#     matches = [n for n in answer if n in correct_numbers]
    
#     rate = len(matches) / N
#     if rate == 1:
#         continue

#     print("音声数:", N)
#     print(f"正答率: {rate*100:.02f}%\n")
    
# print("Done")